In [2]:
# A notebook to try out code that acts to extract variance.

In [26]:
import os
import pandas as pd
import pybedtools
import pybedtools.featurefuncs as featurefuncs
import numpy as np

In [4]:
EPIGENOME_GROUPS = ["embryonic_facial_prominence", "forebrain", "heart", "hindbrain",
                   "intestine", "kidney", "limb", "liver", "lung", "midbrain", "neural_tube", "stomach"]

FEATURES = ["CTCF_Binding_Site", "Enhancer", "Promoter", "TF_Binding_Site"]

In [33]:
EPIGENOME_GROUPS = ["embryonic_facial_prominence"]

FEATURES = ["CTCF_Binding_Site"]

In [5]:
def generate_counts(epigenome_bed, intervals_bed, window_size):
    # Defining names & centre of each IAP
    names = ["chrom", "start", "end", "element_id", "length", "strand", "hits"]
    interval_center = intervals_bed.each(
        featurefuncs.center, width=10).saveas()

    # Finding hits
    center_slop = interval_center.slop(b=window_size, genome="mm10")
    counts = center_slop.intersect(epigenome_bed, c=True)
    hits_df = counts.to_dataframe(names=names)
    hits_df.index = hits_df["element_id"].astype(int).to_list()

    return hits_df["hits"]

In [6]:
# Loading datasets:
dataset_list = os.listdir("data/biomart_data/")
total_df = pd.read_pickle("data/labelled_iaps.pkl")
interval_bed = pybedtools.BedTool(
    "data/clean_beds/mm10.IAP.mended.extent.bed")

In [24]:
total_df.shape[0]

8059

In [34]:
i = 0
for genomic_feature in FEATURES:
    
    gen_str = len(genomic_feature)
    feature_filtered = [
        x for x in dataset_list if x[0:gen_str] == genomic_feature]
    
    for epigenome_group in EPIGENOME_GROUPS:
        # Filtering filename list:
        tot_str = gen_str + len(epigenome_group)
        epigenome_feature_filtered = [
            x for x in feature_filtered if x[gen_str+1:tot_str+1] == epigenome_group]

        # Initialising count array:
        count_array = np.zeros((total_df.shape[0], 1))

        # Generating & allocating counts:
        for epigenome_name in epigenome_feature_filtered:
            dataset = pd.read_pickle("data/biomart_data/" + epigenome_name)
            dataset["chromosome_name"] = [
                "chr" + str(x) for x in list(dataset["chromosome_name"])]
            dataset_bed = pybedtools.BedTool.from_dataframe(dataset)
            
            counts_instance = generate_counts(
                dataset_bed, interval_bed, 10000).to_numpy()
            counts_instance = np.expand_dims(counts_instance, 1)
            count_array = np.concatenate((count_array, counts_instance), axis=1)

        # Computing variance over count array:
        var_array = np.var(count_array, axis=1)
        
        # Formatting & assignment:
        var_series = pd.Series(var_array)
        var_series.index = total_df["element_id"].astype(int).to_list()
        column_id = genomic_feature + "_" + epigenome_group + "_variance"
        total_df[column_id] = var_series

In [35]:
total_df

,chrom,start,end,element_id,length,strand,val_result,CTCF_Binding_Site_embryonic_facial_prominence_variance
151177,chr1,95057294,95057326,151177,33,-,-1,0.122449
25136,chr1,20055335,20055371,25136,37,-,-1,0.122449
159399,chr1,100567693,100567739,159399,47,+,-1,0.000000
160006,chr1,100931364,100931413,160006,50,+,-1,0.000000
91370,chr1,60586644,60586694,91370,51,-,-1,0.122449
...,...,...,...,...,...,...,...,...
4180871,chr9,92734665,92742098,4180871,7434,-,-1,0.489796
4041241,chr9,19763109,19770697,4041241,7589,-,-1,0.000000
4069873,chr9,36157171,36165366,4069873,8196,+,-1,0.000000
4041384,chr9,19897067,19906574,4041384,9508,+,-1,0.000000


In [ ]:
        # Loading epigenome data:
        dataset = pd.read_pickle("data/biomart_data/" + filename)
        dataset["chromosome_name"] = [
            "chr" + str(x) for x in list(dataset["chromosome_name"])]
        dataset_bed = pybedtools.BedTool.from_dataframe(dataset)
        
        
        # Generating counts:
        feature_id = filename[:-4]
        total_df[feature_id + "_counts_100"] = generate_counts(
            dataset_bed, interval_bed, 100)
        total_df[feature_id + "_counts_1k"] = generate_counts(
            dataset_bed, interval_bed, 1000)
        total_df[feature_id + "_counts_5k"] = generate_counts(
            dataset_bed, interval_bed, 5000)
        total_df[feature_id + "_counts_10k"] = generate_counts(
            dataset_bed, interval_bed, 10000)
        total_df[feature_id + "_counts_50k"] = generate_counts(
            dataset_bed, interval_bed, 50000)

In [ ]:
for filename in dataset_list:
    # Generating progress indicator:
    if i % 20 == 0:
        print(str(i) + " datasets processed.")
    i += 1

    # Loading epigenome data:
    dataset = pd.read_pickle("data/biomart_data/" + filename)
    dataset["chromosome_name"] = [
        "chr" + str(x) for x in list(dataset["chromosome_name"])]
    dataset_bed = pybedtools.BedTool.from_dataframe(dataset)

    # Generating counts:
    feature_id = filename[:-4]
    total_df[feature_id + "_counts_100"] = generate_counts(
        dataset_bed, interval_bed, 100)
    total_df[feature_id + "_counts_1k"] = generate_counts(
        dataset_bed, interval_bed, 1000)
    total_df[feature_id + "_counts_5k"] = generate_counts(
        dataset_bed, interval_bed, 5000)
    total_df[feature_id + "_counts_10k"] = generate_counts(
        dataset_bed, interval_bed, 10000)
    total_df[feature_id + "_counts_50k"] = generate_counts(
        dataset_bed, interval_bed, 50000)

total_df.to_pickle("data/iap_counts.pkl")